<a href="https://colab.research.google.com/github/ColeLeal/NFL_Data_Scrape/blob/main/Nfl_dataScrape_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import csv

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Make a list of the title of each row that we want

df_Cols = ['team_name','season_year','game_date','game_outcome','team_record','opponent','pts_off',
            'pts_def','first_down_off','yards_off','pass_yds_off','rush_yds_off','to_off',
            'first_down_def','yards_def','pass_yds_def','rush_yds_def','to_def']
#gameData = df_Cols

In [ ]:
# Specify starting url
urlpage1 = 'https://www.pro-football-reference.com/years/2010/games.htm'

In [ ]:
# Request main page of interest
from urllib.request import Request, urlopen
req = Request(urlpage1, headers = {'User-Agent': 'XYZ/3.0'})
webpage1 = urlopen(req).read()

soup = BeautifulSoup(webpage1, 'html.parser')


seasonGameExts = []
# For each season page we get every teams link ext (for that season) and add the ext to an array
for i in range(11):
  # Link to each seasons page
  if i > 0:
    # Get next season link extension 
    seasonResult = soup.find('div',attrs={'class': 'prevnext'}).find('a',attrs={'class':'button2 next'}).get('href')
    seasonRequest = Request('https://www.pro-football-reference.com'+seasonResult)
    seasonPage = urlopen(seasonRequest).read()
    soup = BeautifulSoup(seasonPage, 'html.parser')

  # Get each winners table cell and each losers data cell for the whole year
  gameResultsW = soup.find_all('td', attrs={'data-stat' : 'winner'})
  gameResultsL = soup.find_all('td', attrs={'data-stat': 'loser'})

  # Append each secondary link from the first 16 (wk1) games of the season
  # This will contain link extensions to every team for that season
  for i in range(16):
    seasonGameExts.append(gameResultsW[i].find('a').get('href'))
    seasonGameExts.append(gameResultsL[i].find('a').get('href'))
  print(len(seasonGameExts))
  print('loop number:',i)

seasonGameExts

32
loop number: 15
64
loop number: 15
96
loop number: 15
128
loop number: 15
160
loop number: 15
192
loop number: 15
224
loop number: 15
256
loop number: 15
288
loop number: 15
320
loop number: 15
352
loop number: 15


['/teams/nor/2010.htm',
 '/teams/min/2010.htm',
 '/teams/pit/2010.htm',
 '/teams/atl/2010.htm',
 '/teams/oti/2010.htm',
 '/teams/rai/2010.htm',
 '/teams/tam/2010.htm',
 '/teams/cle/2010.htm',
 '/teams/mia/2010.htm',
 '/teams/buf/2010.htm',
 '/teams/chi/2010.htm',
 '/teams/det/2010.htm',
 '/teams/nwe/2010.htm',
 '/teams/cin/2010.htm',
 '/teams/htx/2010.htm',
 '/teams/clt/2010.htm',
 '/teams/jax/2010.htm',
 '/teams/den/2010.htm',
 '/teams/nyg/2010.htm',
 '/teams/car/2010.htm',
 '/teams/crd/2010.htm',
 '/teams/ram/2010.htm',
 '/teams/gnb/2010.htm',
 '/teams/phi/2010.htm',
 '/teams/sea/2010.htm',
 '/teams/sfo/2010.htm',
 '/teams/was/2010.htm',
 '/teams/dal/2010.htm',
 '/teams/rav/2010.htm',
 '/teams/nyj/2010.htm',
 '/teams/kan/2010.htm',
 '/teams/sdg/2010.htm',
 '/teams/gnb/2011.htm',
 '/teams/nor/2011.htm',
 '/teams/phi/2011.htm',
 '/teams/ram/2011.htm',
 '/teams/rav/2011.htm',
 '/teams/pit/2011.htm',
 '/teams/jax/2011.htm',
 '/teams/oti/2011.htm',
 '/teams/htx/2011.htm',
 '/teams/clt/201

In [ ]:
%%time
for seasonGameExt in seasonGameExts:
  # Request secondary page
  req2 = Request('https://www.pro-football-reference.com'+seasonGameExt)
  page2 = urlopen(req2).read()
  soup2 = BeautifulSoup(page2, 'html.parser')

  # Get season year and current team of interests name
  data_temp = soup2.find('div',attrs={'data-template':'Partials/Teams/Summary'}).find_all('span')
  gameYear = data_temp[0].getText()
  currentTeam_Name = data_temp[1].getText()

  # Contains all games for team and game stats
  table2 = soup2.find('div', attrs={'id':'all_games'}).find('tbody')
  
  # Get every row in the current table
  results2 = table2.find_all('tr')

  # Loop through each element in the row
  for result in results2:
    cols = result.find_all('td')
    gameDate = cols[1].getText()
    gameOutcome = cols[4].getText()
    team_record = cols[6].getText()
    opp_name = cols[8].getText()
    points_scored = cols[9].getText()
    points_opp = cols[10].getText()
    first_down = cols[11].getText()
    yards_off = cols[12].getText()
    pass_yds_off = cols[13].getText()
    rush_yds_off = cols[14].getText()
    to_off = cols[15].getText()
    first_down_def = cols[16].getText()
    yards_def = cols[17].getText()
    pass_def = cols[18].getText()
    rush_def = cols[19].getText()
    to_def = cols[20].getText()
    gameData.append([currentTeam_Name,gameYear,gameDate,gameOutcome,team_record,opp_name,points_scored,
                    points_opp,first_down,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,
                    yards_def,pass_def,rush_def,to_def]) 

CPU times: user 19min 26s, sys: 4.87 s, total: 19min 31s
Wall time: 24min 31s


In [ ]:
# Print length of data
len(gameData[17:])


6384

In [ ]:
import pandas as pd 
gameData_df = pd.DataFrame(gameData[18:],columns=df_Cols)

In [ ]:
# Show top and bottom of data 
display(gameData_df.head(10))
display(gameData_df.tail(10))

,team_name,season_year,game_date,game_outcome,team_record,opponent,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def
0,New Orleans Saints,2010,September 9,W,1-0,Minnesota Vikings,14,9,18,308,229,79,,12,253,162,91,1
1,New Orleans Saints,2010,September 20,W,2-0,San Francisco 49ers,25,22,17,287,237,50,,24,417,275,142,4
2,New Orleans Saints,2010,September 26,L,2-1,Atlanta Falcons,24,27,20,398,355,43,3,25,417,215,202,1
3,New Orleans Saints,2010,October 3,W,3-1,Carolina Panthers,16,14,27,383,262,121,2,10,251,133,118,1
4,New Orleans Saints,2010,October 10,L,3-2,Arizona Cardinals,20,30,19,358,273,85,4,12,194,153,41,1
5,New Orleans Saints,2010,October 17,W,4-2,Tampa Bay Buccaneers,31,6,27,475,263,212,1,21,277,235,42,1
6,New Orleans Saints,2010,October 24,L,4-3,Cleveland Browns,17,30,25,394,336,58,4,12,210,85,125,
7,New Orleans Saints,2010,October 31,W,5-3,Pittsburgh Steelers,20,10,19,318,288,30,2,13,279,171,108,2
8,New Orleans Saints,2010,November 7,W,6-3,Carolina Panthers,34,3,27,408,243,165,1,15,195,68,127,2
9,New Orleans Saints,2010,,,,Bye Week,,,,,,,,,,,,


,team_name,season_year,game_date,game_outcome,team_record,opponent,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def
6373,Denver Broncos,2020,November 1,W,3-4,Los Angeles Chargers,31,30,17,351,243,108,1,28,485,275,210,2
6374,Denver Broncos,2020,November 8,L,3-5,Atlanta Falcons,27,34,23,405,302,103,1,23,363,271,92,1
6375,Denver Broncos,2020,November 15,L,3-6,Las Vegas Raiders,12,37,20,313,247,66,5,21,357,154,203,
6376,Denver Broncos,2020,November 22,W,4-6,Miami Dolphins,20,13,19,459,270,189,2,17,223,167,56,1
6377,Denver Broncos,2020,November 29,L,4-7,New Orleans Saints,3,31,6,112,12,100,3,19,292,63,229,1
6378,Denver Broncos,2020,December 6,L,4-8,Kansas City Chiefs,16,22,19,330,151,179,2,25,447,313,134,
6379,Denver Broncos,2020,December 13,W,5-8,Carolina Panthers,32,27,23,365,269,96,1,22,370,245,125,
6380,Denver Broncos,2020,December 19,L,5-9,Buffalo Bills,19,48,19,255,115,140,1,29,534,352,182,1
6381,Denver Broncos,2020,December 27,L,5-10,Los Angeles Chargers,16,19,23,396,263,133,2,16,316,227,89,
6382,Denver Broncos,2020,January 3,L,5-11,Las Vegas Raiders,31,32,24,446,324,122,,22,465,359,106,4


In [ ]:
# Save df to csv
gameData_df.to_csv('Nfl_Game_Data_v2.csv')

In [ ]:
len(gameData[18:])

6383